In [28]:
from sklearn.datasets import make_classification
import pandas as pd 
import numpy as np 

In [29]:
X, y = make_classification(n_samples=400, n_features=4)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3', 'f4'])
y = pd.Series(y)

Нужно написать метд `.fit(X, y)`

In [374]:
class MyTreeClf:
    class Node: 
        def __init__(self, y, depth=0, idx=None): 
            self.feature = None                   # по какой фиче разделяем
            self.threshold = None                 # порог разбиения 
            self.left = None                      # левый дочерний узел 
            self.right = None                     # правый дочерний узел 
            self.prediction = None                # значение предсказания, если это лист
            self.is_leaf = False                  # является ли узел листом 
            self.y = y                            # таргеты, относящиеся к узлу 
            self.depth = depth                    # глубина узла 
            self.idx = idx                        # индексы строк X|y, принадлежащие к этому узлу
            
    def __init__(self, max_depth=5, min_samples_split=2, max_leafs=20): 
        self.max_depth = max_depth                      # максимальная глубина дерева
        self.min_samples_split = min_samples_split      # минимальное число элементов в листе
        self.max_leafs = max_leafs                      # максимальное число листов дерева
        self.leafs_cnt = 0                              # счетчик листьев дерева 
        
    def __repr__(self): 
        return f'MyTreeClf class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}'

    def __entropy(self, y): 
        y = y.copy()
        p = np.array([x/len(y) for x in np.bincount(y)])  # считаем вероятность вхождения каждого класса в сплит
        p = p[p > 0]                                      # убираем 0 вероятности
        return np.sum(-p*np.log2(p))

    def __information_gain(self, y, y_left, y_right): 
        # Размеры выборок
        n, n_left, n_right = len(y), len(y_left), len(y_right)
        # Если одна из подвыборок пуста — сплит бесполезен, прирост информации равен 0
        if n_left == 0 or n_right == 0: 
            return 0
        # Энтропия исходной выборки (до разделения)
        H_parent = self.__entropy(y)
    
        # Энтропии левой и правой подвыборок (после разделения)
        H_left = self.__entropy(y_left)
        H_right = self.__entropy(y_right)
    
        # Прирост информации = уменьшение энтропии после разбиения
        IG = H_parent - (n_left / n) * H_left - (n_right / n) * H_right
    
        # Возвращаем значение прироста информации
        return IG    

    def __get_best_split(self, X, y):
        # Создаём копии X и y, чтобы не изменять исходные данные
        X, y = X.copy(), y.copy()
   
        # Глобальные переменные для хранения лучшего сплита по всему датасету
        best_IG = -np.inf           # Максимальный прирост информации среди всех колонок
        best_col = None             # Название колонки, по которой достигается лучший сплит
        best_split_value = None     # Значение сплита, которое даёт этот прирост
        
        # Проходим по каждой колонке признаков
        for col in X.columns: 
            # Берём уникальные значения текущей колонки и сортируем их
            unique_values = np.sort(X[col].unique())
        
            # Проходим по всем парам соседних уникальных значений для формирования порогов
            for i in range(len(unique_values)-1): 
                # Порог для сплита — среднее между двумя соседними уникальными значениями
                split = (unique_values[i] + unique_values[i+1]) / 2  
            
                # Создаём булевы маски для левой и правой подвыборки
                left_mask = X[col] <= split
                right_mask = X[col] > split
            
                # Формируем соответствующие подвыборки таргета
                y_left, y_right = y[left_mask], y[right_mask]

                # Вычисляем прирост информации для текущего сплита
                IG = self.__information_gain(y, y_left, y_right)
            
                # Если этот сплит даёт лучший прирост информации среди всех колонок — обновляем глобальные переменные
                if IG > best_IG:
                    best_IG = IG
                    best_split_value = split
                    best_col = col
    
        # Возвращаем кортеж с лучшей колонкой, значением сплита и максимальным приростом информации
        return best_col, best_split_value, best_IG

    def fit(self, X, y): 
        X, y = X.copy(), y.copy() 
        root = self.Node(y=y, depth=0, idx=X.index)
        leafs = [root]     # cписок узлов
        while leafs: 
            leaf = leafs.pop(-1)
            X_leaf, y_leaf = X.loc[leaf.idx], y.loc[leaf.idx]

            if (leaf.depth >= self.max_depth or
                len(y_leaf) < self.min_samples_split or
                len(set(y_leaf)) == 1 or 
                self.leafs_cnt + 1 >= self.max_leafs):
                leaf.prediction = y_leaf.mean()  # самый частый класс
                leaf.is_leaf = True 
                self.leafs_cnt += 1 
                continue                # идем к следующему узлу 
            # находим лучший сплит для текущего узла
            col, threshold, ig  = self.__get_best_split(X_leaf, y_leaf)  
            # если прирост информации нулевой, делаем лист 
            if ig == 0: 
                leaf.prediction = y_leaf.mean() 
                leaf.is_leaf = True
                self.leafs_cnt += 1 
                continue 
            # проверяем, что хватит мества для двух новых листьев
            if self.leafs_cnt + 2 > self.max_leafs: 
                leaf.prediction = y_leaf.mean()
                leaf.is_leaf = True
                self.leafs_cnt += 1 
                continue
            # делим данные по порогу
            left_mask = X_leaf[col] <= threshold     
            right_mask = X_leaf[col] > threshold
            left_idx = X_leaf[left_mask].index
            right_idx = X_leaf[right_mask].index 
            
            # создаем дочерние узлы
            left_node = self.Node(y=y.loc[left_idx], depth=leaf.depth + 1, idx=left_idx) 
            right_node = self.Node(y=y.loc[right_idx], depth=leaf.depth + 1, idx=right_idx)
            
            # присваиваем родителю данные сплита 
            leaf.left, leaf.right = left_node, right_node 
            leaf.feature = col 
            leaf.threshold = threshold
            
            # добавляем в сипсок листьев 
            leafs.append(left_node)
            leafs.append(right_node)
    
        self.root = root   

    def print_tree(self):
        def _print(node, depth=0):
            indent = "  " * depth  # создаём отступ в зависимости от глубины
            if node.is_leaf:
                print(f"{indent}leaf = {node.prediction}")
            else:
                print(f"{indent}{node.feature} > {node.threshold}")
                if node.left is not None:
                    _print(node.left, depth + 1)
                if node.right is not None:    
                    _print(node.right, depth + 1)
        _print(self.root)

                
            

In [376]:
df = pd.read_csv('banknote+authentication.zip', header=None)
df.columns = ['variance', 'skewness', 'curtosis', 'entropy', 'target']
X, y = df.iloc[:,:4], df['target']

In [379]:
tree_1 = MyTreeClf(max_depth=1, min_samples_split=1, max_leafs=2)

In [381]:
tree_1

MyTreeClf class: max_depth=1, min_samples_split=1, max_leafs=2

In [383]:
tree_1.fit(X, y)

In [384]:
tree_1.print_tree()

variance > 0.320165
  leaf = 0.8112633181126332
  leaf = 0.1076923076923077


In [372]:
model.leafs_cnt

5

In [391]:
tree_2 = MyTreeClf(max_depth=3, min_samples_split=2, max_leafs=5)

In [393]:
tree_2

MyTreeClf class: max_depth=3, min_samples_split=2, max_leafs=5

In [395]:
tree_2.fit(X, y)

In [396]:
tree_2.print_tree()

variance > 0.320165
  leaf = 0.8112633181126332
  variance > 1.7907000000000002
    curtosis > -2.2721999999999998
      leaf = 0.9473684210526315
      leaf = 0.10227272727272728
    curtosis > -4.802
      leaf = 0.6
      leaf = 0.0041928721174004195
